In [55]:
from __future__ import print_function
import numpy as np
# import tensorflow as tf
import keras
# import matplotlib.pyplot as plt
# %matplotlib inline
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
import pickle
from sklearn.linear_model import LogisticRegression
from keras import regularizers

import keras.backend as K

from model import Model
K.clear_session()

In [72]:
num_classes = 10
ns = 10 # number of shadow models
ds = 15000 # number of data points per shadow model for training
batch_size = 128
epochs = 100
lrate = 0.001
decay = 1e-7

In [57]:
X_attack = np.load('data/X_attack.npy')
y_attack = np.load('data/y_attack.npy')
y_attack_not_one_hot = y_attack.copy()
y_attack = keras.utils.to_categorical(y_attack, num_classes)

print(X_attack.shape, y_attack.shape)

(30000, 28, 28, 1) (30000, 10)


In [58]:
X_train_attack = np.zeros((2*ds*ns,num_classes))
y_train_attack = np.zeros((2*ds*ns,))
labels_attack_train = np.zeros((2*ds*ns,num_classes))

for j in np.arange(ns):
    
    indices = np.arange(X_attack.shape[0])
    np.random.seed(j)
    np.random.shuffle(indices)
    X_member_shadow = X_attack[indices[:ds]]
    y_member_shadow = y_attack[indices[:ds]]
    X_nonmember_shadow = X_attack[indices[ds:2*ds]]
    y_nonmember_shadow = y_attack[indices[ds:2*ds]]
    
    shadow = Model(input_shape=X_attack.shape[1:], num_classes=num_classes)
    shadow.train(X_member_shadow, y_member_shadow, batch_size, epochs, lrate, decay, X_nonmember_shadow, y_nonmember_shadow)
    print('\nFor shadow model %d'%j)
    shadow.eval()
    model_name = 'mnist_shadow_'+str(ds)+'_'+str(j)+'.h5'
    shadow.save("models/shadow_models/"+model_name)
    
    
    y_pred_member = shadow.predict(X_member_shadow)
    y_pred_nonmember = shadow.predict(X_nonmember_shadow)
    X_train_attack[j*2*ds:(j+1)*2*ds] = np.vstack((y_pred_member,y_pred_nonmember))
    y_train_attack[j*2*ds:(2*j+1)*ds] = 1
    labels_attack_train[j*2*ds:(j+1)*2*ds] = np.vstack((y_member_shadow,y_nonmember_shadow))

X_attack_train_dict = {'X_train_attack':X_train_attack,'y_train_attack':y_train_attack,'labels_attack_train':labels_attack_train}
fname = 'data/attack_train_data_'+str(ds)
np.save(fname,X_attack_train_dict)

2022-11-25 10:50:28.600637: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 10:50:28.601679: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.



For shadow model 0
Training accuracy = 1.000000
Validation accuracy = 0.954000
47/47 [==============================] - 1s 20ms/step

For shadow model 1
Training accuracy = 1.000000
Validation accuracy = 0.951333
47/47 [==============================] - 1s 28ms/step

For shadow model 2
Training accuracy = 1.000000
Validation accuracy = 0.954667
47/47 [==============================] - 1s 11ms/step

For shadow model 3
Training accuracy = 1.000000
Validation accuracy = 0.951333
47/47 [==============================] - 1s 16ms/step

For shadow model 4
Training accuracy = 1.000000
Validation accuracy = 0.960667
47/47 [==============================] - 1s 22ms/step

For shadow model 5
Training accuracy = 1.000000
Validation accuracy = 0.958000
47/47 [==============================] - 1s 10ms/step

For shadow model 6
Training accuracy = 1.000000
Validation accuracy = 0.955333
47/47 [==============================] - 1s 10ms/step

For shadow model 7
Training accuracy = 1.000000
Validation ac

In [19]:
C_test = [0.1,1,10]
gam_test = [0.001,0.01,0.1]
bestCgam = []

In [59]:
def getfilename(datatype,ds):
    if datatype == 'train':
        name = 'data/attack_'+datatype+'_data_'+str(ds)+'.npy'
    elif datatype == 'test': 
        if use_condensed:
            name = 'data/'+str(ipc)+'/attack_'+datatype+'_data_condensed.npy'
        else:
            name = 'data/attack_'+datatype+'_data.npy'
    elif datatype == 'results':
        if use_condensed:
            name = 'results/attack_'+datatype+'_'+str(ds)+'_'+str(ipc)+'ipc_condensed.npy'
        else:
            name = 'results/attack_'+datatype+'_'+str(ds)+'.npy'
    return name

def getmodelname(classi,ds):
    name = 'models/attack_models/sklearn/attack_model_'+str(ds)+'_class_'+str(classi)+'.p'
    return name

In [60]:
X_train_list = []
y_train_list = []

train_dict = np.load(getfilename('train',ds), allow_pickle=True).item()


labels_attack_train = train_dict['labels_attack_train']
X_train_attack = train_dict['X_train_attack']
y_train_attack = train_dict['y_train_attack']
# print(test_dict)

np.random.seed(123)
index = np.arange(X_train_attack.shape[0])
np.random.shuffle(index)

X_train_attack = X_train_attack[index]
y_train_attack = y_train_attack[index]

for i in range(num_classes):
    ind_train = np.where(labels_attack_train[:,i]==1)[0]
    X_train_list.append(X_train_attack[ind_train])
    y_train_list.append(y_train_attack[ind_train])

class_c_g = np.zeros((10,2))
prec = []
recall = []
acc_classwise = []

for i in range(num_classes):
    x = X_train_list[i]
    y = y_train_list[i]

    # model = LogisticRegression(random_state=0, max_iter=500).fit(x, y)

    # modelname = getmodelname(i,ds)
    # with open( modelname, "wb" ) as fp:
    #     pickle.dump(model, fp)
    
    nfold = 5
    kf = KFold(n_splits=nfold, shuffle=True)
    acc = np.zeros((3,3,nfold))
    for ifold, ind in enumerate(kf.split(x)):
        # Get the training data in the split
        Itr,Its = ind
        xtr = x[Itr,:]
        ytr = y[Itr]
        xts = x[Its,:]
        yts = y[Its]
        for ic,c in enumerate(C_test):
            for ig,g in enumerate(gam_test):
                svc = svm.SVC(probability=False,  kernel="rbf", C=c, gamma=g,verbose=10)
                svc.fit(xtr,ytr)
                yhat_ts = svc.predict(xts)
                acc[ic,ig,ifold] = np.mean(yhat_ts == yts)
    
    acc1 = np.mean(acc,axis=2)
    ci = np.argmax(np.amax(acc1,axis=1))
    gi = np.argmax(np.amax(acc1,axis=0))
    class_c_g[i,0] = C_test[ci]
    class_c_g[i,1] = gam_test[gi]
    
    #Now creating the actual attack classifier for class i and data_size ds
    svc = svm.SVC(probability=False,  kernel="rbf", C=C_test[ci], gamma=gam_test[gi], verbose=10)
    svc.fit(x,y)
    
    
    modelname = getmodelname(i,ds)
    with open( modelname, "wb" ) as fp:
        pickle.dump( [svc, C_test[ci], gam_test[gi]], fp)

In [76]:
X_test_list = []
y_test_list = []
use_condensed = True
ipc=10

test_dict = np.load(getfilename('test',ds), allow_pickle=True).item()

labels_attack_test = test_dict['labels_attack_test']
X_test_attack = test_dict['X_attack_test']
y_test_attack = test_dict['y_attack_test']

for i in range(num_classes):
    
    ind_test = np.where(labels_attack_test[:,i]==1)[0]
    X_test_list.append(X_test_attack[ind_test])
    y_test_list.append(y_test_attack[ind_test])

yhat_full = np.zeros(y_test_attack.shape[0])
y_full = np.zeros(y_test_attack.shape[0])

start = 0

for i in range(num_classes):
    
    xtest = X_test_list[i]
    ytest = y_test_list[i]
    ntest = xtest.shape[0]

    # modelname = getmodelname(i,ds)
    # with open( modelname, "rb" ) as fp:
    #     model = pickle.load(fp)

    modelname = getmodelname(i,ds)
    with open( modelname, "rb" ) as fp:
        svc = pickle.load(fp)[0]

    yhat_test = model.predict(xtest)
    yhat_test[yhat_test>0.5] = 1
    yhat_test[yhat_test<=0.5] = 0
    acci = np.mean(yhat_test == ytest)
    preci,reci,_,_= precision_recall_fscore_support(ytest,yhat_test,average='binary')
    prec.append(preci)
    recall.append(reci)
    acc_classwise.append(acci)
    y_full[start:(start+ntest)] = ytest
    yhat_full[start:(start+ntest)] = yhat_test
    print(f"For class {i}:")
    print(f"\t precision: {preci}")
    print(f"\t recall: {reci}")
    print(f"\t accuracy: {acci}")
    start = start+ntest

acctotal = np.mean(yhat_full == y_full)
prectotal,recalltotal,_,_= precision_recall_fscore_support(y_full,yhat_full,average='binary')
fname = getfilename('results',ds)
dict_to_save = {'prectotal':prectotal,
                'acctotal':acctotal,
                'recalltotal':recalltotal,
                'prec':prec,
                'acc':acc_classwise,
                'recall':recall,
                'class_c_g':class_c_g}
np.save(fname,dict_to_save)
print('For ds = ',ds)
print('Attack Precision: ',prectotal)
print('Attack Recall: ',recalltotal)
print('Attack Accuracy: ',acctotal)

For class 0:
	 precision: 0.5144539614561028
	 recall: 0.6406666666666667
	 accuracy: 0.5116514690982776
For class 1:
	 precision: 0.0
	 recall: 0.0
	 accuracy: 0.4972246567338592
For class 2:
	 precision: 0.5021682567215958
	 recall: 0.39986187845303867
	 accuracy: 0.5105156037991859
For class 3:
	 precision: 0.5057471264367817
	 recall: 0.26018396846254926
	 accuracy: 0.5076472502440612
For class 4:
	 precision: 0.515625
	 recall: 0.11141120864280891
	 accuracy: 0.49726589200273413
For class 5:
	 precision: 0.5490196078431373
	 recall: 0.041822255414488425
	 accuracy: 0.5094130675526024
For class 6:
	 precision: 0.4949253731343284
	 recall: 0.562415196743555
	 accuracy: 0.4988235294117647
For class 7:
	 precision: 0.4915151515151515
	 recall: 0.5165605095541401
	 accuracy: 0.48567750241390406
For class 8:
	 precision: 0.5193798449612403
	 recall: 0.13968033356497567
	 accuracy: 0.504695652173913
For class 9:
	 precision: 0.4740484429065744
	 recall: 0.09096945551128818
	 accuracy: 0.

/home/eshan.rg/.conda/envs/privacy/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
